In [1]:
%matplotlib qt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg as la
from scipy import fft 
from scipy import linalg as la
from scipy import signal
from scipy import optimize
from scipy.optimize import curve_fit
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
import pywt


In [2]:
#Il dataframe finale è 'data'
DataFrame=pd.read_csv("center_sternum.txt",sep=None,engine='python')
DataFrame['time_s'] = np.arange(0,DataFrame.shape[0])/200 # let's construct the time axis, with each sampling separated by a 200th of a second (because sampling rate is 200 Hz)
DataFrame=DataFrame.set_index('time_s')

df = DataFrame.drop(columns=['Log Mode','Log Freq','Timestamp','MagnX','MagnY','MagnZ','qw','qi','qj','qk'])
data = df.loc[9:71].copy()
print(data.columns)

Index(['AccX', 'AccY', 'AccZ', 'GyroX', 'GyroY', 'GyroZ'], dtype='object')


In [3]:
#scaling data
dataS = pd.DataFrame(MinMaxScaler().fit_transform(data))
dataS.columns = ['AccXS', 'AccYS', 'AccZS', 'GyroXS', 'GyroYS', 'GyroZS']
#print(dataS)
#dataS.plot(y = ['AccXS', 'AccYS', 'AccZS'])
accS_columns = ['AccXS', 'AccYS', 'AccZS']
gyrS_columns = ['GyroXS', 'GyroYS', 'GyroZS']

In [4]:
#PCA on accS_columns
pca=PCA(n_components=1) # we seek a single signal
# X must be (n_samples,n_features)
pca.fit(dataS[accS_columns])
print("Percentage of variance due to the selected component: {:.2%}".format(pca.explained_variance_ratio_[0]))
Acc = pca.transform(dataS[accS_columns]).reshape(-1)

Percentage of variance due to the selected component: 55.64%


In [5]:
#PCA on gyrS_columns
pca.fit(dataS[gyrS_columns])
print("Percentage of variance due to the selected component: {:.2%}".format(pca.explained_variance_ratio_[0]))
Gyr = pca.transform(dataS[gyrS_columns]).reshape(-1)

Percentage of variance due to the selected component: 61.90%


In [15]:
#%matplotlib notebook
#9-th order wavelet transform and fourier transform over Acc
db4 = pywt.Wavelet('db4')
n = len(Acc)

cA9,cD9,cD8,cD7, cD6,cD5, cD4, cD3, cD2, cD1 = pywt.wavedec(Acc, db4, level=9)
coefD9 = pywt.upcoef('d',cD9,'db4',take=n, level=9) #sto ricostruendo il segnale partendo esclusivamenente dai coefficienti
                                                    #details del livello 9. Se ho capito bene, questa è la cosa giusta da fare, veder anche che succede se metto 'a' anzi che 'd'

fft_coefD9 = fft.fft(coefD9)                          #trasformata di fourier del livello 9
freq = fft.fftfreq(coefD9.size, d = 1/200)
mask = (freq > 0) & (freq < 0.75)
power_coefD9 = np.abs(fft_coefD9)[mask]
freq = freq[mask]

fft_Acc = fft.fft(Acc)                          #trasformata di fourier di Acc
power_Acc = np.abs(Acc)[mask]

fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(figsize = (6,6), nrows = 2, ncols = 2)

ax1.plot(coefD9, color = 'black')
ax3.plot(freq,power_coefD9, color = 'pink')
ax2.plot(Acc, color = 'green')
ax4.plot(freq,power_Acc, color = 'red')

ax1.set_title('9-th level signal')
ax2.set_title('Acc')
ax3.set_title('9-th level FT')
ax4.set_title('Acc FT')

ax3.grid(True,linestyle='--')
ax4.grid(True,linestyle='--')

plt.tight_layout()
plt.show();

print(coefD9.size,Acc.size)
print(freq[power_coefD9 == max(power_coefD9)])

12401 12401
[0.22578824]


In [7]:
#faccio lo stesso su Gyr
cA9,cD9,cD8,cD7, cD6,cD5, cD4, cD3, cD2, cD1=pywt.wavedec(Gyr, db4, level=9)
coefD9 = pywt.upcoef('d',cD9,'db4',take=n, level=9) #sto ricostruendo il segnale partendo esclusivamenente dai coefficienti
                                                    #details del livello 9. Se ho capito bene, questa è la cosa giusta da fare, veder anche che succede se metto 'a' anzi che 'd'

fft_coefD9 = fft.fft(coefD9)                          #trasformata di fourier del livello 9
freq = fft.fftfreq(coefD9.size, d = 1/200)
mask = (freq > 0) & (freq < 1.5)
power_coefD9 = np.abs(fft_coefD9)[mask]
freq = freq[mask]

fft_Acc = fft.fft(Gyr)                          #trasformata di fourier di Gyr
power_Acc = np.abs(Gyr)[mask]

fig, ((ax1,ax2),(ax3,ax4)) = plt.subplots(figsize = (6,6), nrows = 2, ncols = 2)
ax1.plot(coefD9, color = 'black')
ax3.plot(freq,power_coefD9, color = 'pink')
ax2.plot(Gyr, color = 'green')
ax4.plot(freq,power_Acc, color = 'red')
plt.tight_layout()
plt.show()

print(freq[power_coefD9 > 30])

[0.19353278 0.22578824 0.29029917]
